<a href="https://colab.research.google.com/github/sinungadi/TwitterABSA/blob/master/sentiment_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install xformers
!pip install numpyencoder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 7.1 MB/s eta 0:00:00


# 1. Import Required Libraries

In [2]:
import pandas as pd
import warnings
import random
import json
from numpyencoder import NumpyEncoder
from tqdm import tqdm

import torch
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from transformers import pipeline
import numpy as np
from scipy.special import softmax

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

# 2. Data Colection

In [3]:
data = pd.read_csv('data_filtered.csv')

In [4]:
docs = data['text_cleaned']

# 3. Create Functions

In [5]:
def getModel(model_name):
  model_name = f"w11wo/indonesian-roberta-base-sentiment-classifier"
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  config = AutoConfig.from_pretrained(model_name)
  model = AutoModelForSequenceClassification.from_pretrained(model_name)

  return tokenizer, config, model

def process(sequence, tokenizer, config, model):
  encoded_input = tokenizer(sequence, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  # Print labels and scores
  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  result = {}
  for i in range(scores.shape[0]):
      l = config.id2label[ranking[i]]
      s = scores[ranking[i]]
      result[l] = s

  return result

def generateTemp():
  return {
      'id' : None,
      'sequence': None,
      'result': []
  }

In [6]:
# Dict to parse the results into each label and class
def parseResultList():
  return {
      'id' : None,
      'sequence': None,
      'sentiment' : None
}

# Get the True False class for each label
def getSentiment(data):
  return max(data['result'], key=data['result'].get)

# 3. Data Modeling

In [7]:
model_name = f"w11wo/indonesian-roberta-base-sentiment-classifier"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 98eda446-ed76-43ef-85cd-a3ef258cd86d)')' thrown while requesting HEAD https://huggingface.co/w11wo/indonesian-roberta-base-sentiment-classifier/resolve/main/merges.txt


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: f4a9fbfb-8315-4d8b-996b-73e07d6bcc52)')' thrown while requesting HEAD https://huggingface.co/w11wo/indonesian-roberta-base-sentiment-classifier/resolve/main/special_tokens_map.json


In [8]:
resultList = []

for i, text in tqdm(enumerate(docs)):
  temp = generateTemp()

  temp.update({
      'id' : data['tweet_id'][i],
      'sequence' : text,
      'result' : process(text,tokenizer,config,model)
  })

  resultList.append(temp)

23729it [57:38,  6.86it/s]


In [9]:
random.sample(resultList, 5)

[{'id': 1635252546225991681,
  'sequence': 'Kalo saja charging station di Indonesia sudah tersedia secara merata, mungkin bakal banyak orang yang mulai berlomba-lomba untuk jadi yang pertama mengelilingi Indonesia dengan kendaraan listrik',
  'result': {'positive': 0.7547986,
   'neutral': 0.13014053,
   'negative': 0.11506089}},
 {'id': 1629376734939471872,
  'sequence': 'Home Charging Produk Layanan PLN Penuhi Kebutuhan Pengisian Baterai Kendaraan Listrik di Rumah: Layanan home charging ini juga tersambung dengan Electric Vehicle Digital Services (EVDS) yang disiapkan PLN.',
  'result': {'neutral': 0.9981945,
   'positive': 0.0010955199,
   'negative': 0.00070995156}},
 {'id': 1626522623776530433,
  'sequence': 'Statemen di atas bilang mendorong penggunaan kendaraan umum dan tidak profesional kendaraan pribadi, tapi di sisi lain adanya subsidi kendaraan listrik mendorong masyarakat untuk membeli kendaraan lagi????',
  'result': {'negative': 0.9846687,
   'neutral': 0.009373622,
   'p

In [10]:
with open('SentClfResult_Indo-Roberta.json', 'w') as file:
    json.dump(resultList, file, indent=4, sort_keys=False,
              separators=(', ', ': '), ensure_ascii=False,
              cls=NumpyEncoder)

In [11]:
resultData = []

for tweet in resultList:
  resultDict = parseResultList()

  resultDict.update({
      'id' : tweet['id'],
      'sequence' : tweet['sequence'],
      'sentiment' : getSentiment(tweet)
  })

  resultData.append(resultDict)

In [12]:
df = pd.DataFrame(resultData)

In [13]:
df.sample(5)

,id,sequence,sentiment
16781,1650534930043240451,"Bangun Pabrik Baterai Mobil Listrik di Kanada, VW Investasi Rp218,4 Triliun",neutral
20321,1660898545136959490,"Perbandingannya cukup jauh, dengan asumsi tarif listrik sebesar Rp1.699 per Kwh, hanya diperlukan sekitar Rp2.500 untuk sepeda motor listrik menempuh jarak 50 kamu dan 10 kamu untuk mobil listrik. Sedangkan, jika menggunakan BBM kamu harus menghabiskan sekitar Rp14 ribu -",positive
8958,1641060532856979458,"motor biasa saja sekali jajan ke bengkel lumayan, lah ini hype buat motor listrik, bengkel tidak jelas dimana, terus di pikir sekali jajan akan murah gitu..",negative
1811,1616541590528593920,"Bro tahu produksi listrik kita masih didominasi pakai batu bara? Pake kendaraan listrik itu tidak ngurangi polusi, cuma mindah polusi dari kota2 besar ke daerah produksi listrik. Kalo kamu pakai istilah distribusi polusi di Indonesia saya baru setuju. Singapura dapat listrik darimana?",negative
13675,1634719522119880704,"Fokus utang luar negeri, IKN, subsidi motor / mobil listrik, Mega proyek Rugi, jalan tol rugi gerus APBN untuk kesejahteraan rakyat duit pajak rakyat !;",negative


In [14]:
df.to_csv('SentClfResult_Indo-Roberta.csv', index=False)